In [1]:
import pandas as pd
import numpy as np
import os

import explore

In [2]:
nbastats, nbapo, nbadata, nbastats_po, pbpstats, shotdetail, shotdetail_po = explore.import_nba()

In [3]:
df = [nbastats, nbapo, nbadata, nbastats_po, 
      pbpstats, shotdetail, shotdetail_po]

# Which features to keep?

To begin I will just be keeping score by first quarter, half, 3rd quarter and final.

Each test will be in the next sequence. The model will try all teh games in the past, and the last 5 to ten games. As well as their last games. 

### Features

team_id
home or away
opponent_id
record total
record 
total points scored in the event
wins over last 5
wins over last game
wins over last 10


##### offensive stats
points
points first quarter
points second quarter
points third quarter
field goal percentage
field goal pts
three point pts
three point percentage
shooting fouls drawn
free throws
free throw percentage
assists


##### defensive stats
blocks 
steals 
turnovers 
rebounds
personal fouls


#### like to have
starting 5
- number of points
- complete prodution broken down by player
bench player
height of starting 5

In [4]:
shotdetail.shape

(217218, 24)

### fields that contain data i need

#### nbastats
GAME_ID
EVENTMSGTYPE
EVENTMSGACTIONTYPE
PERIOD
HOMEDESCRIPTION
NEUTRALDESCRIPTION
VISITORDESCRIPTION
SCORE
PLAYER1_TEAM_ID
PLAYER1_TEAM_CITY
PLAYER1_TEAM_NICKNAME
PLAYER2_TEAM_ID
PLAYER2_TEAM_CITY
PLAYER2_TEAM_NICKNAME
PLAYER3_TEAM_ID
PLAYER3_TEAM_CITY
PLAYER3_TEAM_NICKNAME



##### drop
EVENTNUM
WCTIMESTRING
PCTIMESTRING
SCOREMARGIN
PERSON1TYPE
PLAYER1_NAME
PLAYER1_TEAM_ABBREVIATION
PERSON2TYPE
PLAYER2_ID
PLAYER2_NAME
PLAYER2_TEAM_ABBREVIATION
PERSON3TYPE
PLAYER3_TEAM_ABBREVIATION
VIDEO_AVAILABLE_FLAG


#### nbadata
evt
de?
opt1 -?
opt2 -?
opt3 -?
opt4 -?
tid
hs
vs
epid - ?
oftid
ord -?
period
game - id

###### drop
wallclk
cl
locx
locy


#### pbpstats
events
fg2a
fg2m
fg3a
fg3m
gamedate -?
gameid
OFFENSIVEREBOUNDS
opponent
period
SHOOTINGFOULSDRAWN
TURNOVERS
DESCRIPTION -?


##### drop
endtime
NONSHOOTINGFOULSTHATRESULTEDINFTS
OFFENSIVEREBOUNDS
STARTSCOREDIFFERENTIAL
STARTTIME
STARTTYPE
URL


#### shotdetail
game_id
TEAM_ID
TEAM_NAME
PERIOD
EVENT_TYPE
ACTION_TYPE
SHOT_TYPE
HTM	

##### drop
grid_type
game_event_id
player_id
PLAYER_NAME
MINUTES_REMAINING
SECONDS_REMAINING
SHOT_ZONE_BASIC
SHOT_ZONE_AREA
SHOT_ZONE_RANGE
SHOT_DISTANCE
LOC_X
LOC_Y
SHOT_ATTEMPTED_FLAG
SHOT_MADE_FLAG
GAME_DATE
VTM

## wrangle.py code

In [5]:
# drop columns
nbastats = nbastats.drop(columns =[
    'EVENTNUM', 'WCTIMESTRING', 'PCTIMESTRING', 'SCOREMARGIN', 
    'PERSON1TYPE', 'PLAYER1_NAME', 'PLAYER1_TEAM_ABBREVIATION',
    'PERSON2TYPE', 'PLAYER2_ID', 'PLAYER2_NAME', 'PLAYER2_TEAM_ABBREVIATION',
    'PERSON3TYPE', 'PLAYER3_TEAM_ABBREVIATION', 'VIDEO_AVAILABLE_FLAG'
                                    ])

In [6]:
# drop columns
nbadata = nbadata.drop(columns=['wallclk','cl','locX','locY'])

In [7]:
# drop columns
pbpstats = pbpstats.drop(columns=[
    'ENDTIME', 'NONSHOOTINGFOULSTHATRESULTEDINFTS', 
    'OFFENSIVEREBOUNDS', 'STARTSCOREDIFFERENTIAL',
    'STARTTIME', 'STARTTYPE', 'URL'
                                ])

In [8]:
# drop columns
shotdetail = shotdetail.drop(columns=[
    'GRID_TYPE', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME', 
    'MINUTES_REMAINING', 'SECONDS_REMAINING', 'SHOT_ZONE_BASIC',
    'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE', 
    'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG',
    'GAME_DATE', 'VTM'
                                    ])

In [9]:
sdf = nbastats, nbadata, pbpstats, shotdetail

In [10]:
for i in sdf:
    i.columns = [col.lower() for col in i.columns]

In [11]:
# rename some columns
pbpstats = pbpstats.rename(columns={'gameid':'game_id'})

In [12]:
## End wrangel py code

In [13]:
sdf = nbastats, nbadata, pbpstats, shotdetail

In [14]:
pbpstats.game_id.min(), pbpstats.game_id.max()

(22200001, 22201230)

In [15]:
nbastats.game_id.min(), nbastats.game_id.max()

(22200001, 22201230)

In [16]:
nbadata.game_id.min(), nbadata.game_id.max()

(22200001, 22201230)

In [17]:
shotdetail.game_id.min(), shotdetail.game_id.max()

(22200001, 22201230)

In [18]:
# Game ID will be my unique key to create a dataframe
# Step 1: Extract unique game IDs from each DataFrame
unique_game_ids = [set(df['game_id'].unique()) for df in sdf]

# Step 2: Identify the common set of game IDs across all DataFrames
common_game_ids = set.intersection(*unique_game_ids)

# Step 3: Check for missing or extra game IDs in each DataFrame
for i, df in enumerate(sdf):
    extra_ids = unique_game_ids[i] - common_game_ids
    missing_ids = common_game_ids - unique_game_ids[i]
    
    if extra_ids:
        print(f"DataFrame {i} has extra game IDs: {extra_ids}")
    if missing_ids:
        print(f"DataFrame {i} is missing game IDs: {missing_ids}")

    

In [19]:
for i in sdf:
    display(i.game_id.nunique())

1230

1230

1230

1230

### Features

team_id
home or away
opponent_id
record total
record 
total points scored in the event
wins over last 5
wins over last game


##### offensive stats
points
field goal percentage
field goal pts
three point pts
three point percentage
shooting fouls drawn
free throws
free throw percentage
assists


##### defensive stats
blocks 
steals 
turnovers 
rebounds
personal fouls

In [20]:
# tool don't include
shotdetail[shotdetail.game_id==22200005]

,game_id,team_id,team_name,period,event_type,action_type,shot_type,htm
0,22200005,1610612737,Atlanta Hawks,1,Made Shot,Pullup Jump shot,2PT Field Goal,ATL
1,22200005,1610612737,Atlanta Hawks,1,Missed Shot,Pullup Jump shot,3PT Field Goal,ATL
2,22200005,1610612737,Atlanta Hawks,1,Missed Shot,Running Jump Shot,3PT Field Goal,ATL
3,22200005,1610612737,Atlanta Hawks,1,Missed Shot,Jump Shot,3PT Field Goal,ATL
4,22200005,1610612737,Atlanta Hawks,1,Missed Shot,Driving Layup Shot,2PT Field Goal,ATL
...,...,...,...,...,...,...,...,...
71966,22200005,1610612745,Houston Rockets,4,Missed Shot,Layup Shot,2PT Field Goal,ATL
71967,22200005,1610612745,Houston Rockets,4,Made Shot,Jump Shot,3PT Field Goal,ATL
71968,22200005,1610612745,Houston Rockets,4,Missed Shot,Reverse Layup Shot,2PT Field Goal,ATL
71969,22200005,1610612745,Houston Rockets,4,Missed Shot,Pullup Jump shot,3PT Field Goal,ATL


In [21]:
# creating final dataframe from team
team = shotdetail[['game_id', 'team_id', 'team_name', 'htm']]

In [22]:
# drop duplicates
team = team.drop_duplicates(subset=['team_id','game_id'])

In [23]:
# tool, don't include
team[team.game_id==22200005]

,game_id,team_id,team_name,htm
0,22200005,1610612737,Atlanta Hawks,ATL
71873,22200005,1610612745,Houston Rockets,ATL


In [24]:
# sorts df by game_id to be used as an index later
team = team.sort_values('game_id')

In [25]:
# code to display complete dataframes
# BE SURE TO RUN RESET AFTER CELL
# Set the maximum number of rows to display
pd.set_option('display.max_rows', None)

# Set the maximum number of columns to display
pd.set_option('display.max_columns', None)

# Set the width of each column to prevent wrapping/truncation of data
pd.set_option('display.width', None)

# Set the maximum column width to display data without truncation
pd.set_option('display.max_colwidth', None)


In [26]:
team.head(80)

,game_id,team_id,team_name,htm
159154,22200001,1610612755,Philadelphia 76ers,BOS
14552,22200001,1610612738,Boston Celtics,BOS
93564,22200002,1610612747,Los Angeles Lakers,GSW
64480,22200002,1610612744,Golden State Warriors,GSW
152080,22200003,1610612753,Orlando Magic,DET
57340,22200003,1610612765,Detroit Pistons,DET
210092,22200004,1610612764,Washington Wizards,IND
79159,22200004,1610612754,Indiana Pacers,IND
0,22200005,1610612737,Atlanta Hawks,ATL
71873,22200005,1610612745,Houston Rockets,ATL


In [27]:
# code resets pandas to truncate dataframes 
# when necesary for display
pd.reset_option('all')


In [28]:
# This dictionary will be used to make a home catagory 
# and an away catagory
team_id_dict = {
    1610612755: "Philadelphia 76ers",
    1610612738: "Boston Celtics",
    1610612747: "Los Angeles Lakers",
    1610612744: "Golden State Warriors",
    1610612753: "Orlando Magic",
    1610612765: "Detroit Pistons",
    1610612764: "Washington Wizards",
    1610612754: "Indiana Pacers",
    1610612737: "Atlanta Hawks",
    1610612745: "Houston Rockets",
    1610612740: "New Orleans Pelicans",
    1610612751: "Brooklyn Nets",
    1610612748: "Miami Heat",
    1610612741: "Chicago Bulls",
    1610612761: "Toronto Raptors",
    1610612739: "Cleveland Cavaliers",
    1610612752: "New York Knicks",
    1610612763: "Memphis Grizzlies",
    1610612760: "Oklahoma City Thunder",
    1610612750: "Minnesota Timberwolves",
    1610612766: "Charlotte Hornets",
    1610612759: "San Antonio Spurs",
    1610612762: "Utah Jazz",
    1610612743: "Denver Nuggets",
    1610612756: "Phoenix Suns",
    1610612742: "Dallas Mavericks",
    1610612757: "Portland Trail Blazers",
    1610612758: "Sacramento Kings",
    1610612749: "Milwaukee Bucks",
    1610612746: "LA Clippers",
}

In [29]:
# htm dictionary to be converted into a team_id number
htm_data = {
    "BOS": 1610612738,
    "GSW": 1610612744,
    "DET": 1610612765,
    "IND": 1610612754,
    "ATL": 1610612737,
    "BKN": 1610612751,
    "MIA": 1610612748,
    "TOR": 1610612761,
    "MEM": 1610612763,
    "MIN": 1610612750,
    "SAS": 1610612759,
    "UTA": 1610612762,
    "PHX": 1610612756,
    "SAC": 1610612758,
    "PHI": 1610612755,
    "LAL": 1610612747,
    "CHA": 1610612766,
    "WAS": 1610612764,
    "HOU": 1610612745,
    "NYK": 1610612752,
    "MIL": 1610612749,
    "DAL": 1610612742,
    "DEN": 1610612743,
    "ORL": 1610612753,
    "CLE": 1610612739,
    "CHI": 1610612741,
    "POR": 1610612757,
    "NOP": 1610612740,
    "LAL": 1610612746,
    "OKC": 1610612760
}


In [30]:
# changes the htm to a number
team['htm'] = team['htm'].replace(htm_data)


In [31]:
# renames to home and visitor
team = team.rename(columns={'team_id':'visitor','htm':'home'})

In [32]:
team

,game_id,visitor,team_name,home
159154,22200001,1610612755,Philadelphia 76ers,1610612738
14552,22200001,1610612738,Boston Celtics,1610612738
93564,22200002,1610612747,Los Angeles Lakers,1610612744
64480,22200002,1610612744,Golden State Warriors,1610612744
152080,22200003,1610612753,Orlando Magic,1610612765
...,...,...,...,...
100765,22201228,1610612747,Los Angeles Lakers,1610612746
93464,22201229,1610612746,LA Clippers,1610612756
173313,22201229,1610612756,Phoenix Suns,1610612756
71777,22201230,1610612744,Golden State Warriors,1610612757


In [33]:
# drops the second copy of the game_id 
team = team.loc[team['visitor'] != team['home']]

In [34]:
team.head()

,game_id,visitor,team_name,home
159154,22200001,1610612755,Philadelphia 76ers,1610612738
93564,22200002,1610612747,Los Angeles Lakers,1610612744
152080,22200003,1610612753,Orlando Magic,1610612765
210092,22200004,1610612764,Washington Wizards,1610612754
71873,22200005,1610612745,Houston Rockets,1610612737


In [35]:
# dropping team_name column
team = team.drop(columns=['team_name'])

In [36]:
# this dataframe in this state is what three days of work look like
team

,game_id,visitor,home
159154,22200001,1610612755,1610612738
93564,22200002,1610612747,1610612744
152080,22200003,1610612753,1610612765
210092,22200004,1610612764,1610612754
71873,22200005,1610612745,1610612737
...,...,...,...
187560,22201227,1610612758,1610612743
209991,22201228,1610612762,1610612746
100765,22201228,1610612747,1610612746
93464,22201229,1610612746,1610612756


### all that is left is to make game_id the index 
### and add other features, 
### and spit into 32 differnet dataframes
-  maybe it would bea good idea to change the team ids into 1-30 based on preseason standings. then thehy will be easiier to get normalized.

In [37]:
team.head()

,game_id,visitor,home
159154,22200001,1610612755,1610612738
93564,22200002,1610612747,1610612744
152080,22200003,1610612753,1610612765
210092,22200004,1610612764,1610612754
71873,22200005,1610612745,1610612737


### end team clean section

In [38]:
nbadata[(nbadata.game_id==22200005) & (nbadata['period'] == 4)]


,evt,de,opt1,opt2,opt3,opt4,mtype,etype,opid,tid,pid,hs,vs,epid,oftid,ord,pts,period,game_id
323304,477,Start Period,0,0,0,0,0,12,NaN,0,0,84,80,NaN,1610612737,4760000,0,4,22200005
323305,478,[ATL] Okongwu Driving Hook Shot: Missed,2,0,0,0,57,2,NaN,1610612737,1630168,84,80,NaN,1610612737,4770000,0,4,22200005
323306,479,[HOU] Eason Rebound (Off:3 Def:4),0,0,0,0,0,4,NaN,1610612745,1631106,84,80,NaN,1610612737,4780000,0,4,22200005
323307,480,[HOU] Sengun Turnover : Bad Pass (4 TO) Steal:...,1,0,0,0,1,5,203200.0,1610612745,1630578,84,80,NaN,1610612745,4790000,0,4,22200005
323308,482,[ATL 87-80] Holiday 3pt Shot: Made (5 PTS) Ass...,3,1,0,0,103,1,NaN,1610612737,1628988,87,80,1627749.0,1610612737,4810000,3,4,22200005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323417,638,[HOU] Porter Jr. Rebound (Off:3 Def:4),1,0,0,0,0,4,NaN,1610612745,1629645,117,107,NaN,1610612745,6340000,0,4,22200005
323418,639,[HOU] Gordon 3pt Shot: Missed,3,0,0,0,1,2,NaN,1610612745,201569,117,107,NaN,1610612745,6350000,0,4,22200005
323419,640,[ATL] Collins Rebound (Off:0 Def:8),0,0,0,0,0,4,NaN,1610612737,1628381,117,107,NaN,1610612745,6360000,0,4,22200005
323420,641,End Period,0,0,0,0,0,13,NaN,0,0,117,107,NaN,1610612737,6370000,0,4,22200005


In [39]:
nbadata.head(2)

,evt,de,opt1,opt2,opt3,opt4,mtype,etype,opid,tid,pid,hs,vs,epid,oftid,ord,pts,period,game_id
0,2,Start Period,0,0,0,0,0,12,NaN,0,0,0,0,NaN,0,20000,0,1,22200001
1,4,Jump Ball Embiid vs Horford (Harris gains poss...,0,0,0,0,0,10,201143.0,1610612755,203954,0,0,202699.0,1610612755,60000,0,1,22200001


In [40]:
# seperating final scores by game_id
final_scores = nbadata.groupby('game_id').agg({'hs':'last', 'vs':'last'}).reset_index()

In [41]:
final_scores

,game_id,hs,vs
0,22200001,126,117
1,22200002,123,109
2,22200003,113,109
3,22200004,107,114
4,22200005,117,107
...,...,...,...
1225,22201226,115,100
1226,22201227,109,95
1227,22201228,128,117
1228,22201229,114,119


In [42]:
# merging final scores onto teams
team = team.merge(final_scores, on='game_id', how='left')

In [43]:
team.head()

,game_id,visitor,home,hs,vs
0,22200001,1610612755,1610612738,126,117
1,22200002,1610612747,1610612744,123,109
2,22200003,1610612753,1610612765,113,109
3,22200004,1610612764,1610612754,107,114
4,22200005,1610612745,1610612737,117,107


In [44]:
# assigns 1 if home team won and 0 if home team lost
team['result'] = np.where(team['hs'] > team['vs'], 1, 0)

In [45]:
team.head()

,game_id,visitor,home,hs,vs,result
0,22200001,1610612755,1610612738,126,117,1
1,22200002,1610612747,1610612744,123,109,1
2,22200003,1610612753,1610612765,113,109,1
3,22200004,1610612764,1610612754,107,114,0
4,22200005,1610612745,1610612737,117,107,1


In [46]:
# set 'game_id' as the index
team.set_index('game_id', inplace=True)

In [47]:
team.head()

,visitor,home,hs,vs,result
game_id,,,,,
22200001,1610612755,1610612738,126,117,1
22200002,1610612747,1610612744,123,109,1
22200003,1610612753,1610612765,113,109,1
22200004,1610612764,1610612754,107,114,0
22200005,1610612745,1610612737,117,107,1


In [48]:
# Initialize the new columns with NaN values
team['hlast'] = np.nan
team['vlast'] = np.nan

# Loop through the DataFrame to populate hlast and vlast
for idx, row in team.iterrows():
    # Get the home and visitor team ids
    home_team = row['home']
    visitor_team = row['visitor']

    # Find the last game for the home team
    prev_home_game = team[((team['home'] == home_team) | (team['visitor'] == home_team)) & (team.index < idx)]
    if not prev_home_game.empty:
        # If home team was the home team in their last game and won
        if (prev_home_game.iloc[-1]['home'] == home_team) and (prev_home_game.iloc[-1]['result'] == 1):
            team.at[idx, 'hlast'] = 1
        # If home team was the visitor team in their last game and lost
        elif (prev_home_game.iloc[-1]['visitor'] == home_team) and (prev_home_game.iloc[-1]['result'] == 0):
            team.at[idx, 'hlast'] = 1
        else:
            team.at[idx, 'hlast'] = 0
    
    # Find the last game for the visitor team
    prev_visitor_game = team[((team['home'] == visitor_team) | (team['visitor'] == visitor_team)) & (team.index < idx)]
    if not prev_visitor_game.empty:
        # If visitor team was the visitor team in their last game and won
        if (prev_visitor_game.iloc[-1]['visitor'] == visitor_team) and (prev_visitor_game.iloc[-1]['result'] == 0):
            team.at[idx, 'vlast'] = 1
        # If visitor team was the home team in their last game and lost
        elif (prev_visitor_game.iloc[-1]['home'] == visitor_team) and (prev_visitor_game.iloc[-1]['result'] == 1):
            team.at[idx, 'vlast'] = 1
        else:
            team.at[idx, 'vlast'] = 0

# filling first game with losses
# it is 16 observations
team['hlast'].fillna(0, inplace=True)
team['vlast'].fillna(0, inplace=True)

# makes columns int, after i drop the nulls         
team['hlast'] = team['hlast'].astype(int)
team['vlast'] = team['vlast'].astype(int)


In [49]:
# Create functions to compute rolling wins for home and visitor
def home_last5_wins(row):
    previous_games = team[(team['home'] == row['home']) & (team.index < row.name)]
    return previous_games['result'].tail(5).sum()

def visitor_last5_wins(row):
    previous_games = team[(team['visitor'] == row['visitor']) & (team.index < row.name)]
    return (1 - previous_games['result']).tail(5).sum()

# Apply the functions row-wise to compute the rolling wins and assign to the desired column names
team['home_last5'] = team.apply(home_last5_wins, axis=1)
team['visitor_last5'] = team.apply(visitor_last5_wins, axis=1)

# Convert to integers
team['home_last5'] = team['home_last5'].astype(int)
team['visitor_last5'] = team['visitor_last5'].astype(int)


In [51]:
team

,visitor,home,hs,vs,result,hlast,vlast,home_last5,visitor_last5
game_id,,,,,,,,,
22200001,1610612755,1610612738,126,117,1,0,0,0,0
22200002,1610612747,1610612744,123,109,1,0,0,0,0
22200003,1610612753,1610612765,113,109,1,0,0,0,0
22200004,1610612764,1610612754,107,114,0,0,0,0,0
22200005,1610612745,1610612737,117,107,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
22201227,1610612758,1610612743,109,95,1,0,0,3,3
22201228,1610612762,1610612746,128,117,1,0,0,3,1
22201228,1610612747,1610612746,128,117,1,0,0,3,3


In [57]:
team.home.unique()

array([1610612738, 1610612744, 1610612765, 1610612754, 1610612737,
       1610612751, 1610612748, 1610612761, 1610612763, 1610612750,
       1610612759, 1610612762, 1610612756, 1610612758, 1610612755,
       1610612746, 1610612766, 1610612764, 1610612752, 1610612745,
       1610612757, 1610612753, 1610612741, 1610612749, 1610612742,
       1610612743, 1610612739, 1610612740, 1610612760, 'LAC'],
      dtype=object)